In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from matplotlib import pyplot as plt

In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()

output_path = Path(os.environ.get("OUTPUT_DIR"))
figures_path = Path(os.environ.get("FIGURES_DIR"))
wave_folder = Path(os.environ.get("WAVE_BENCHMARK_DIR"))
wave_folder

In [ ]:
#target_hdf_advection = "1D_Advection_Sols_beta0.4.hdf5"
target_beta_advection = 0.4
# target_hdf_darcy = "2D_DarcyFlow_beta1.0.hdf5"
target_beta_darcy = 1.0

In [ ]:
advection_path = output_path / "1d_advection"
advection_fv_path = advection_path / "fv" / "results"
advection_col_path = advection_path / "col" / "results"

darcy_path = output_path / "2d_darcy"
darcy_fv_path = darcy_path / "fv" / "results"
darcy_col_path = darcy_path / "col" / "results"

In [ ]:
import yaml

def get_dfs(results_path, beta, problem_type):
    args_paths = results_path.glob("**/args.yaml")
    N_pdes = []
    dfs = []
    for p in args_paths:
        with open(p, "r") as f:
            args = yaml.safe_load(f)
            if args["beta"] != beta:
                continue
            if problem_type == "advection":
                N_pde = args["N_pde_t"] * args["N_pde_x"]
            elif problem_type == "darcy":
                N_pde = args["N_pde_xy"]**2
            else:
                raise ValueError("Unknown problem")
            N_bc = args["N_bc"]
            N_pdes.append(N_pde)
        csv_path = p.with_name("results.csv")
        df = pd.read_csv(csv_path)
        dfs.append(df)
    order = np.argsort(N_pdes)
    N_pdes = [N_pdes[i] for i in order]
    dfs = [dfs[i] for i in order]
    return N_pdes, dfs

In [ ]:
N_pdes_advection_fv, dfs_advection_fv = get_dfs(advection_fv_path, target_beta_advection, "advection")
N_pdes_advection_col, dfs_advection_col = get_dfs(advection_col_path, target_beta_advection, "advection")

N_pdes_darcy_fv, dfs_darcy_fv = get_dfs(darcy_fv_path, target_beta_darcy, "darcy")
N_pdes_darcy_col, dfs_darcy_col = get_dfs(darcy_col_path, target_beta_darcy, "darcy")

In [ ]:
def get_L_infs(dfs):
    return np.array([df["L_inf_err"].mean() for df in dfs])

def get_L_inf_stds(dfs):
    return np.array([np.std(df["L_inf_err"]) for df in dfs])
    # return [df["L_inf_err"].std() for df in dfs]

def get_log_L_infs(dfs):
    return np.array([np.mean(np.log10(df["L_inf_err"])) for df in dfs])

def get_log_L_inf_stds(dfs):
    return np.array([np.std(np.log10(df["L_inf_err"])) for df in dfs])

def get_RMSEs(dfs):
    return [np.sqrt(df["MSE"]).mean() for df in dfs]

def get_RMSE_stds(dfs):
    return np.array([np.sqrt(df["MSE"]).std() for df in dfs])

def get_log_RMSEs(dfs):
    return np.array([np.mean(np.log10(np.sqrt(df["MSE"]))) for df in dfs])

def get_log_RMSE_stds(dfs):
    return np.array([np.std(np.log10(np.sqrt(df["MSE"]))) for df in dfs])

L_infs_advection_fv = get_L_infs(dfs_advection_fv)
L_infs_advection_col = get_L_infs(dfs_advection_col)
L_infs_stds_advection_fv = get_L_inf_stds(dfs_advection_fv)
L_infs_stds_advection_col = get_L_inf_stds(dfs_advection_col)

RMSEs_advection_fv = get_RMSEs(dfs_advection_fv)
RMSEs_advection_col = get_RMSEs(dfs_advection_col)

L_infs_darcy_fv = get_L_infs(dfs_darcy_fv)
L_infs_darcy_col = get_L_infs(dfs_darcy_col)
L_infs_stds_darcy_fv = get_L_inf_stds(dfs_darcy_fv)
L_infs_stds_darcy_col = get_L_inf_stds(dfs_darcy_col)

RMSEs_darcy_fv = get_RMSEs(dfs_darcy_fv)
RMSEs_darcy_col = get_RMSEs(dfs_darcy_col)

In [ ]:
wave_df_fv = pd.read_csv(wave_folder / "errs_fv.csv")
wave_df_col = pd.read_csv(wave_folder / "errs_col.csv")

wave_df_fv = wave_df_fv.drop(wave_df_fv.columns[0], axis='columns')
wave_df_col = wave_df_col.drop(wave_df_col.columns[0], axis='columns')
max_depth = 5
num_dimensions = 3

levels = np.arange(-1, max_depth+1)
num_obs = lambda level: 0 if level == -1 else (2**num_dimensions) ** level
num_obs = np.vectorize(num_obs)

fv_vals = [float(x) for x in wave_df_fv.values.flatten()]
col_vals = [float(x) for x in wave_df_col.values.flatten()]
# Construct points for scatter plot
tile_factor = int(len(fv_vals) / len(levels))
points_fv = [np.tile(num_obs(levels), tile_factor), fv_vals]
points_col = [np.tile(num_obs(levels), tile_factor), col_vals]

fv_means = list(wave_df_fv.mean())
fv_stds = list(wave_df_col.std())

col_means = list(wave_df_col.mean())
col_stds = list(wave_df_col.std())

fv_errs = np.array(fv_stds)
col_errs = np.array(col_stds)

In [ ]:
def plot_advection(ax, N_pdes_fv, N_pdes_col, errs_fv, errs_col, PINN_err=None, 
                   UNet_err=None, PINN_label_pos=None, UNet_label_pos=None, err_label="RMSE"):
    ax.plot(N_pdes_fv, errs_fv, ".-", label="GP-FVM")
    ax.plot(N_pdes_col, errs_col, ".-", label="Collocation")
    if PINN_err is not None:
        ax.axhline(y=PINN_err, linestyle="--", label="PINN", color="C2")
        if PINN_label_pos is None:
            PINN_label_pos = (4**(5.6), PINN_err)
        ax.text(*PINN_label_pos, "PINN", color="C2", ha="center")
    if UNet_err is not None:
        ax.axhline(y=UNet_err, linestyle="--", label="UNet", color="C3")
        if UNet_label_pos is None:
            UNet_label_pos = (4**(5.6), UNet_err)
        ax.text(*UNet_label_pos, "U-Net", color="C3", ha="center")
    ax.set_xscale("log", base=4)
    ax.set_yscale("log", base=10)
    ax.set_xlabel("$N_{\mathrm{PDE}}$")
    ax.set_ylabel(err_label)
    return ax

def log_plot_advection(ax, N_pdes_fv, N_pdes_col, dfs_fv, dfs_col, err_label="MAE"):
    if err_label == "RMSE":
        log_errs_fv = get_log_RMSEs(dfs_fv)
        log_errs_col = get_log_RMSEs(dfs_col)
        log_err_stds_fv = get_log_RMSE_stds(dfs_fv)
        log_err_stds_col = get_log_RMSE_stds(dfs_col)
    elif err_label == "MAE":
        log_errs_fv = get_log_L_infs(dfs_fv)
        log_errs_col = get_log_L_infs(dfs_col)
        log_err_stds_fv = get_log_L_inf_stds(dfs_fv)
        log_err_stds_col = get_log_L_inf_stds(dfs_col)
    
    ax.plot(N_pdes_fv, 10**log_errs_fv, ".-", label="GP-FVM")
    ax.plot(N_pdes_col, 10**log_errs_col, ".-", label="Collocation")
    ax.fill_between(N_pdes_fv, 10**(log_errs_fv - 1.96 * log_err_stds_fv), 10**(log_errs_fv + 1.96 * log_err_stds_fv), alpha=0.3, color="C0")
    ax.fill_between(N_pdes_col, 10**(log_errs_col - 1.96 * log_err_stds_col), 10**(log_errs_col + 1.96 * log_err_stds_col), alpha=0.3, color="C1")

    ax.set_xscale("log", base=4)
    ax.set_yscale("log", base=10)
    ax.set_xlabel("$N_{\mathrm{PDE}}$")
    ax.set_ylabel(err_label)
    return ax

In [ ]:
def plot_advection_from_beta(ax, beta, PINN_err=None, UNet_err=None, PINN_label_pos=None, UNet_label_pos=None, err_label="Maximum absolute error"):
    N_pdes_fv, dfs_fv = get_dfs(advection_fv_path, beta, "advection")
    N_pdes_col, dfs_col = get_dfs(advection_col_path, beta, "advection")
    if err_label == "RMSE":
        errs_fv = get_RMSEs(dfs_fv)
        errs_col = get_RMSEs(dfs_col)
    elif err_label == "Maximum absolute error":
        errs_fv = get_L_infs(dfs_fv)
        errs_col = get_L_infs(dfs_col)
    return plot_advection(ax, N_pdes_fv, N_pdes_col, errs_fv, errs_col, PINN_err, UNet_err, PINN_label_pos, UNet_label_pos, err_label)

def log_plot_advection_from_beta(ax, beta, err_label):
    N_pdes_fv, dfs_fv = get_dfs(advection_fv_path, beta, "advection")
    N_pdes_col, dfs_col = get_dfs(advection_col_path, beta, "advection")
    return log_plot_advection(ax, N_pdes_fv, N_pdes_col, dfs_fv, dfs_col, err_label)

In [ ]:
from tueplots import bundles
plt.rcParams.update(bundles.neurips2024(nrows=1, ncols=2))

fig, ax = plt.subplots(1, 2)
PINN_RMSE = 4e-1
plot_advection_from_beta(ax[0], 1.0, PINN_err=PINN_RMSE, PINN_label_pos=(4**3, PINN_RMSE - 0.6e-1), err_label="RMSE")
ax[0].set_title("\\textbf{a)} RMSE")
ax[0].grid()
plot_advection_from_beta(ax[1], 1.0, err_label="Maximum absolute error")
ax[1].set_title("\\textbf{b)} Maximum absolute error")
ax[1].grid()
fig.suptitle("1D Advection ($\\beta=1.0$)")
fig.savefig(f"{figures_path}/advection_1.0_RMSE_MAE.pdf")

In [ ]:
from tueplots import bundles
plt.rcParams.update(bundles.neurips2024(nrows=1, ncols=1))

fig, ax = plt.subplots()
plot_advection_from_beta(ax, 0.4, err_label="Maximum absolute error")
ax.grid()
fig.suptitle("1D Advection ($\\beta=0.4$)")
fig.savefig(f"{figures_path}/advection_0.4_MAE.pdf")

In [ ]:
def plot_darcy(ax, N_pdes_fv, N_pdes_col, errs_fv, errs_col, UNet_err=None, UNet_label_pos=None, err_label="RMSE"):
    ax.plot(N_pdes_fv, errs_fv, ".-", label="GP-FVM")
    ax.plot(N_pdes_col, errs_col, ".-", label="Collocation")
    if UNet_err is not None:
        ax.axhline(y=UNet_err, linestyle="--", label="UNet", color="C3")
        if UNet_label_pos is None:
            UNet_label_pos = (4**(5.6), UNet_err)
        ax.text(*UNet_label_pos, "UNet", color="C3", ha="center")
    ax.set_xscale("log", base=4)
    ax.set_yscale("log", base=10)
    ax.set_xlabel("$N_{\mathrm{PDE}}$")
    ax.set_ylabel(err_label)
    return ax

def log_plot_darcy(ax, N_pdes_fv, N_pdes_col, dfs_fv, dfs_col, err_label="MAE"):
    if err_label == "MAE":
        log_errs_fv = get_log_L_infs(dfs_fv)
        log_errs_col = get_log_L_infs(dfs_col)
        log_err_stds_fv = get_log_L_inf_stds(dfs_fv)
        log_err_stds_col = get_log_L_inf_stds(dfs_col)
    elif err_label == "RMSE":
        log_errs_fv = get_log_RMSEs(dfs_fv)
        log_errs_col = get_log_RMSEs(dfs_col)
        log_err_stds_fv = get_log_RMSE_stds(dfs_fv)
        print(log_err_stds_fv)
        log_err_stds_col = get_log_RMSE_stds(dfs_col)
    ax.plot(N_pdes_fv, 10**log_errs_fv, ".-", label="GP-FVM")
    ax.plot(N_pdes_col, 10**log_errs_col, ".-", label="Collocation")
    ax.fill_between(N_pdes_fv, 10**(log_errs_fv - 1.96 * log_err_stds_fv), 10**(log_errs_fv + 1.96 * log_err_stds_fv), alpha=0.3, color="C0")
    ax.fill_between(N_pdes_col, 10**(log_errs_col - 1.96 * log_err_stds_col), 10**(log_errs_col + 1.96 * log_err_stds_col), alpha=0.3, color="C1")
    ax.set_xscale("log", base=4)
    ax.set_yscale("log", base=10)
    ax.set_xlabel("$N_{\mathrm{PDE}}$")
    ax.set_ylabel(err_label)
    return ax

def log_plot_darcy_from_beta(ax, beta, err_label="MAE"):
    N_pdes_fv, dfs_fv = get_dfs(darcy_fv_path, beta, "darcy")
    N_pdes_col, dfs_col = get_dfs(darcy_col_path, beta, "darcy")
    return log_plot_darcy(ax, N_pdes_fv, N_pdes_col, dfs_fv, dfs_col, err_label)

def plot_darcy_from_beta(ax, beta, UNet_err=None, UNet_label_pos=None, err_label="Maximum absolute error"):
    N_pdes_fv, dfs_fv = get_dfs(darcy_fv_path, beta, "darcy")
    N_pdes_col, dfs_col = get_dfs(darcy_col_path, beta, "darcy")
    if err_label == "RMSE":
        errs_fv = get_RMSEs(dfs_fv)
        errs_col = get_RMSEs(dfs_col)
    elif err_label == "Maximum absolute error":
        errs_fv = get_L_infs(dfs_fv)
        errs_col = get_L_infs(dfs_col)
    return plot_darcy(ax, N_pdes_fv, N_pdes_col, errs_fv, errs_col, UNet_err, UNet_label_pos, err_label)

In [ ]:
# fig, ax = plt.subplots()
# log_plot_darcy_from_beta(ax, 1.0, "MAE")

In [ ]:
from tueplots import bundles
plt.rcParams.update(bundles.neurips2024(nrows=1, ncols=2))

fig, ax = plt.subplots(1, 2)
# PINN_RMSE = 4e-1
plot_darcy_from_beta(ax[0], 0.1, err_label="RMSE")
ax[0].set_title("\\textbf{a)} RMSE")
ax[0].grid()
plot_darcy_from_beta(ax[1], 0.1, err_label="Maximum absolute error")
ax[1].set_title("\\textbf{b)} Maximum absolute error")
ax[1].grid()
fig.suptitle("2D Darcy Flow ($\\beta=0.1$)")
fig.savefig(f"{figures_path}/darcy_0.1_RMSE_MAE.pdf")

In [ ]:
from tueplots import bundles
plt.rcParams.update(bundles.neurips2024(nrows=1, ncols=2))

fig, ax = plt.subplots(1, 2)
# PINN_RMSE = 4e-1
plot_darcy_from_beta(ax[0], 0.01, err_label="RMSE")
ax[0].set_title("\\textbf{a)} RMSE")
ax[0].grid()
plot_darcy_from_beta(ax[1], 0.01, err_label="Maximum absolute error")
ax[1].set_title("\\textbf{b)} Maximum absolute error")
ax[1].grid()
fig.suptitle("2D Darcy Flow ($\\beta=0.01$)")
fig.savefig(f"{figures_path}/darcy_0.01_RMSE_MAE.pdf")

In [ ]:
from tueplots import bundles
plt.rcParams.update(bundles.neurips2024())

fig, ax = plt.subplots()
plot_darcy_from_beta(ax, 1.0, err_label="Maximum absolute error")
ax.grid()
fig.suptitle("2D Darcy Flow ($\\beta=1.0$)")
fig.savefig(f"{figures_path}/darcy_1.0_MAE.pdf")

In [ ]:
from tueplots import bundles

plt.rcParams.update(bundles.neurips2024(nrows=1, ncols=2))
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Set1.colors)

fig, ax = plt.subplots(1, 3)

# Advection
plot_advection(ax[0], N_pdes_advection_fv, N_pdes_advection_col, RMSEs_advection_fv, RMSEs_advection_col,
               PINN_err=9.2e-1, UNet_err=3.6e-1, PINN_label_pos=(4**(5.6), 6.5e-1), UNet_label_pos=(4**(5.6), 4.1e-1))
ax[0].set_title("\\textbf{a)} 1D Advection ($\\beta=0.4$)")
ax[0].grid()

# Darcy
plot_darcy_from_beta(ax[1], 1.0, err_label="RMSE")
ax[1].set_title("\\textbf{b)} 2D Darcy Flow ($\\beta=1.0$)")
ax[1].grid()

# Wave
ax[2].plot(num_obs(levels), fv_means, ".-", label="GP-FVM")
ax[2].plot(num_obs(levels), col_means, ".-", label="Collocation")
ax[2].grid()

ax[2].set_xscale('symlog', base=(2**num_dimensions))
ax[2].set_yscale('log')
ax[2].set_xlabel('$N_{\mathrm{PDE}}$')
ax[2].set_title("\\textbf{c)} 2D Wave Equation")
ax[2].set_ylabel("MAE")

fig.savefig(f"{figures_path}/advection_darcy_benchmark.pdf")

In [ ]:
from tueplots import bundles

plt.rcParams.update(bundles.neurips2024(nrows=1, ncols=2))
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Set1.colors)

fig, ax = plt.subplots(1, 3)

# Advection
log_plot_advection_from_beta(ax[0], 0.4, "RMSE")

# ax[0].legend()
ax[0].set_title("\\textbf{a)} 1D Advection ($\\beta=0.4$)")
ax[0].grid()

# Darcy
log_plot_darcy_from_beta(ax[1], 1.0, "RMSE")
ax[1].set_title("\\textbf{b)} 2D Darcy Flow ($\\beta=1.0$)")
ax[1].grid()

# Wave
ax[2].plot(num_obs(levels), fv_means, ".-", label="GP-FVM")
ax[2].plot(num_obs(levels), col_means, ".-", label="Collocation")

# Scatter plot
ax[2].scatter(*points_fv, marker=".", color="C0", alpha=0.4)
ax[2].scatter(*points_col, marker=".", color="C1", alpha=0.4)

ax[2].set_xscale('symlog', base=(2**num_dimensions))
ax[2].set_yscale('log')
ax[2].set_xlabel('$N_{\mathrm{PDE}}$')
ax[2].set_title("\\textbf{c)} 2D Wave Equation")
ax[2].set_ylabel("MAE")

fig.savefig(f"{figures_path}/advection_darcy_benchmark_error_bars.pdf")

In [ ]:
np.log(dfs_advection_fv[-6]["L_inf_err"]).hist()